<a href="https://colab.research.google.com/github/YichengShen/cis5220-project/blob/main/gpt3.5_turbo_with_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-3.5-turbo

In [ ]:
!pip install langchain
!pip install gpt_index

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import argparse
import json
import logging
import os
import re
import shutil

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.schema import BaseLanguageModel
from sqlalchemy import create_engine
from tqdm import tqdm

from gpt_index import GPTSQLStructStoreIndex, LLMPredictor, SQLDatabase, ServiceContext

In [ ]:
os.environ["OPENAI_API_KEY"] = '' # You need to add an API key to run this code.

In [ ]:
# Create data folder if not exist
!mkdir -p data

# Change this path to where you store spider.zip in your Drive
dataset_zip_path_in_drive = "/content/drive/Shareddrives/CIS 522/spider.zip"
dataset_zip_path_in_runtime = "/content/data/spider.zip"

shutil.copy(dataset_zip_path_in_drive, dataset_zip_path_in_runtime)

'/content/data/spider.zip'

In [ ]:
!unzip -q -o /content/data/spider.zip -d /content/data/

In [ ]:
logging.getLogger("root").setLevel(logging.WARNING)

In [ ]:
_spaces = re.compile(r"\s+")
_newlines = re.compile(r"\n+")

In [ ]:
table_paths = "/content/data/spider/tables.json"

if not isinstance(table_paths, list):
        table_paths = (table_paths, )

for i, TABLE_PATH in enumerate(table_paths):
    print(f"Loading data from {TABLE_PATH}")
    with open(TABLE_PATH) as inf:
        table_data= json.load(inf)

Loading data from /content/data/spider/tables.json


In [ ]:
def format_dict(input_dict):
    formatted_value = []

    for i in range(len(input_dict['table_names'])):
        table_name = input_dict['table_names'][i]
        columns = [col[1].replace(" ", "_") for col in input_dict['column_names'] if col[0] == i]
        formatted_columns = ', '.join(columns)
        formatted_value.append(f"{table_name} : {formatted_columns}")

    formatted_value_str = " | ".join(formatted_value)
    return {input_dict['db_id']: formatted_value_str}

formatted_table_data = [format_dict(d) for d in table_data]
merged_formatted_table_data = {k: v for d in formatted_table_data for k, v in d.items()}

In [ ]:
def _generate_sql(
    llama_index: GPTSQLStructStoreIndex,
    nl_query_text: str,
) -> str:
    """Generate SQL query for the given NL query text."""
    
    response = llama_index.query(nl_query_text)
    if (
        response.extra_info is None
        or "sql_query" not in response.extra_info
        or response.extra_info["sql_query"] is None
    ):
        raise RuntimeError("No SQL query generated.")
    query = response.extra_info["sql_query"]
    # Remove newlines and extra spaces.
    query = _newlines.sub(" ", query)
    query = _spaces.sub(" ", query)
    return query.strip()

In [ ]:
def generate_sql(llama_indexes: dict, examples: list, output_file: str) -> None:
    """Generate SQL queries for the given examples and write them to the output file."""
    with open(output_file, "w") as f:
        for example in tqdm(examples, desc=f"Generating {output_file}"):
            db_name = example["db_id"]
            nl_query_text = example["question"] 
            # + " | " + merged_formatted_table_data[db_name]
            # print(nl_query_text)
            # break
            try:
                sql_query = _generate_sql(llama_indexes[db_name], nl_query_text)
            except Exception as e:
                print(
                    f"Failed to generate SQL query for question: "
                    f"{example['question']} on database: {example['db_id']}."
                )
                print(e)
                sql_query = "ERROR"
            f.write(sql_query + "\n")

In [ ]:
# Define variables or use input prompts to get values for arguments
input_path = "/content/data/spider"
output_path = "/content/drive/Shareddrives/CIS 522/GPT3.5_pred_no_col_name"
model_choice = "gpt-3.5-turbo"  # Replace with desired model option

In [ ]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
# Load the Spider dataset from the input directory.
with open(os.path.join(input_path, "train_spider.json"), "r") as f:
    train_spider = json.load(f)
with open(os.path.join(input_path, "train_others.json"), "r") as f:
    train_others = json.load(f)
with open(os.path.join(input_path, "dev.json"), "r") as f:
    dev = json.load(f)

In [ ]:
# Create all necessary SQL database objects.
databases = {}
for db in train_spider + train_others + dev:
    db_name = db["db_id"]
    if db_name in databases:
        continue
    db_path = os.path.join(input_path, "database", db_name, db_name + ".sqlite")
    engine = create_engine("sqlite:///" + db_path)
    databases[db_name] = (SQLDatabase(engine=engine), engine)

/usr/local/lib/python3.9/dist-packages/langchain/sql_database.py:90: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  self._metadata.reflect(
/usr/local/lib/python3.9/dist-packages/langchain/sql_database.py:90: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(
/usr/local/lib/python3.9/dist-packages/gpt_index/langchain_helpers/sql_wrapper.py:26: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self.metadata_obj.reflect()
/usr/local/lib/python3.9/dist-packages/gpt_index/langchain_helpers/sql_wrapper.py:26: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  self.metadata_obj.reflect()
/usr/local/lib/python3.9/dist-packages/langchain

In [ ]:
# Create the LlamaIndexes for all databases.
if model_choice in ["gpt-3.5-turbo", "gpt-4"]:
    llm: BaseLanguageModel = ChatOpenAI(model=model_choice, temperature=0)
else:
    llm = OpenAI(model=model_choice, temperature=0)
llm_predictor = LLMPredictor(llm=llm)
llm_indexes = {}
for db_name, (db, engine) in databases.items():
    # Get the name of the first table in the database.
    # This is a hack to get a table name for the index, which can use any
    # table in the database.
    table_name = engine.execute(
        "select name from sqlite_master where type = 'table'"
    ).fetchone()[0]
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
    llm_indexes[db_name] = GPTSQLStructStoreIndex.from_documents(
        documents=[],
        service_context=service_context,
        sql_database=db,
        table_name=table_name,
    )

/usr/local/lib/python3.9/dist-packages/gpt_index/langchain_helpers/sql_wrapper.py:53: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  for column in self._inspector.get_columns(table_name):
/usr/local/lib/python3.9/dist-packages/gpt_index/langchain_helpers/sql_wrapper.py:53: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  for column in self._inspector.get_columns(table_name):
/usr/local/lib/python3.9/dist-packages/gpt_index/langchain_helpers/sql_wrapper.py:57: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  for foreign_key in self._inspector.get_foreign_keys(table_name):
/usr/local/lib/python3.9/dist-packages/gpt_index/langchain_helpers/sql_wrapper.py:57: SAWarning: WARNING: SQL-parsed foreign key constraint '('Cus

In [ ]:
# llm_indexes
sample_dev = dev[:3]
sample_dev

query = [a["query"] for a in sample_dev]
query

['SELECT count(*) FROM singer',
 'SELECT count(*) FROM singer',
 'SELECT name ,  country ,  age FROM singer ORDER BY age DESC']

In [ ]:
merged_formatted_table_data['perpetrator']

'perpetrator : perpetrator_id, people_id, date, year, location, country, killed, injured | people : people_id, name, height, weight, home_town'

In [ ]:
# Generate SQL queries.
# generate_sql(
#     llama_indexes=llm_indexes,
#     examples=train_spider + train_others,
#     output_file=os.path.join(output_path, "train_pred.sql"),
# )
generate_sql(
    llama_indexes=llm_indexes,
    examples=dev,
    output_file=os.path.join(output_path, "dev_pred.sql"),
)

Generating /content/drive/Shareddrives/CIS 522/GPT3.5_pred_no_col_name/dev_pred.sql:   9%|▉         | 91/1034 [02:27<24:14,  1.54s/it]

Failed to generate SQL query for question: For each continent, list its id, name, and how many countries it has? on database: car_1.
(sqlite3.OperationalError) no such column: c.ContId
[SQL: SELECT c.ContId, c.CountryName, COUNT(*) AS num_countries
FROM countries c
JOIN continents con ON c.Continent = con.ContId
GROUP BY c.Continent
ORDER BY num_countries DESC]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Generating /content/drive/Shareddrives/CIS 522/GPT3.5_pred_no_col_name/dev_pred.sql:  10%|▉         | 100/1034 [02:41<25:42,  1.65s/it]

Failed to generate SQL query for question: Find the name of the makers that produced some cars in the year of 1970? on database: car_1.
(sqlite3.OperationalError) no such column: car_names.MakeId
[SQL: SELECT DISTINCT car_makers.Maker
FROM car_makers
JOIN cars_data ON car_makers.Id = car_names.MakeId
WHERE cars_data.Year = 1970
ORDER BY car_makers.Maker ASC]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Generating /content/drive/Shareddrives/CIS 522/GPT3.5_pred_no_col_name/dev_pred.sql:  10%|▉         | 101/1034 [02:43<26:51,  1.73s/it]

...

Generating /content/drive/Shareddrives/CIS 522/GPT3.5_pred_no_col_name/dev_pred.sql:  87%|████████▋ | 899/1034 [22:04<03:57,  1.76s/it]

Failed to generate SQL query for question: Show the ids of high schoolers who have friends and are also liked by someone else. on database: network_1.
(sqlite3.OperationalError) near "Answer": syntax error
[SQL: SELECT DISTINCT f.student_id
FROM Friend f
JOIN Likes l ON f.student_id = l.liked_id
Answer: The query returns the IDs of high schoolers who have friends and are also liked by someone else.]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Generating /content/drive/Shareddrives/CIS 522/GPT3.5_pred_no_col_name/dev_pred.sql:  87%|████████▋ | 900/1034 [22:06<04:20,  1.94s/it]

Failed to generate SQL query for question: What are the ids of students who both have friends and are liked? on database: network_1.
(sqlite3.OperationalError) 1st ORDER BY term does not match any column in the result set
[SQL: SELECT DISTINCT f.student_id 
FROM Friend f 
JOIN Likes l ON f.student_id = l.student_id 
INTERSECT 
SELECT DISTINCT f.friend_id 
FROM Friend f 
JOIN Likes l ON f.friend_id = l.liked_id
ORDER BY student_id ASC]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Generating /content/drive/Shareddrives/CIS 522/GPT3.5_pred_no_col_name/dev_pred.sql: 100%|██████████| 1034/1034 [25:26<00:00,  1.48s/it]


Evaluation is done in other notebooks.